<img style="float: left;" src="earth-lab-logo-rgb.png" width="150" height="150" />

# Earth Analytics Education - Climate 101 Workshop

<div class='notice--success' markdown="1">

## <i class="fa fa-ship" aria-hidden="true"></i>  NETCDF 4 Climate Data in Open Source Python 

In this lesson, you will learn how to work with Climate Data Sets (MACA v2 for the United states) stored in netcdf 4 format using open source **Python**.

## <i class="fa fa-graduation-cap" aria-hidden="true"></i> Learning Objectives

After completing this lesson, you will be able to:

* Summarize MACA v 2 climate data stored in netcdf 4 format by seasons across all time periods using `xarray`.
* Summarize MACA v 2 climate data stored in netcdf 4 format by seasons and across years using `xarray`.

## <i class="fa fa-check-square-o fa-2" aria-hidden="true"></i> What You Need

You will need a computer with internet access to complete this lesson and the earth-analytics-python
conda environment installed.

</div>

## Subset MACA 2 Climate Data in NetCDF 4 Format By Time and Spatial Extents

In this lesson, you will learn how to subset MACA v2 climate data using the open 
source Python packages `xarray` and `regionmask`.

In [ ]:
# Make sure ea dir exists - if not create it
import warnings
import os
import earthpy as et
ea_path = os.path.join(et.io.HOME, 'earth-analytics', 'data')

if not os.path.exists(ea_path):
    os.makedirs(ea_path)

warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
import geopandas as gpd
import earthpy as et
import xarray as xr
import netCDF4
import regionmask


# Plotting options
sns.set(font_scale=1.3)
sns.set_style("white")

# Optional - set your working directory if you wish to use the data
# accessed lower down in this notebook (the USA state boundary data)
os.chdir(os.path.join(et.io.HOME, 'earth-analytics', 'data'))

## Spatial Subsets of Data Using an AOI

In the previous lesson, you learned how to select a single point and extract 
temperature data at that location. You can also create areas of interest 
(AOIs) that define the geographic region that you wish to extract data for. 

Below you will learn how to use a shapefile that contains a boundary
area of interest that you wish to subset data for. You will then learn 
how to subset the data using the 

1. geographic boundary extent of the AOI (a rectangular extent) and
2. using the actual shape boundary of the AOI (example: the state of California)

To begin, you will open up a new netcdf file. This file contains 
future projected maximum temperature data for the Continental United States(CONUS).  

In [ ]:
# Get netcdf file
data_path_monthly = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_macav2metdata_tasmax_BNU-ESM_r1i1p1_rcp45_2006_2099_CONUS_monthly.nc'

# Open up the data
with xr.open_dataset(data_path_monthly) as file_nc:
    monthly_forecast_temp_xr = file_nc

# View xarray object
monthly_forecast_temp_xr

### Open A Shapefile to Use as an AOI

Often you want to subset and summarize climate data by specific regions of 
interest. In the example below, you will open a natural earth layer that 
contains state and region boundaries. You will extract a state boundary (California)
within the United States to use as an AOI.

In [ ]:
# Download natural earth data which contains state boundaries to generate AOI
et.data.get_data(
    url="https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/cultural/ne_50m_admin_1_states_provinces_lakes.zip")

states_path = "earthpy-downloads/ne_50m_admin_1_states_provinces_lakes"
states_path = os.path.join(
    states_path, "ne_50m_admin_1_states_provinces_lakes.shp")

states_gdf = gpd.read_file(states_path)
states_gdf.head()

In [ ]:
# You will use the bounds to determine the slice values for this data
# Select any state in the CONUS that you wish here! California is the default
cali_aoi = states_gdf[states_gdf.name == "California"]
# Get the total spatial extent for California
cali_aoi.total_bounds

Next, convert the bounds of your AOI into the min and max longitude
and latitude values. You will use these values to `slice` your data.

In [ ]:
# Get lat min, max
aoi_lat = [float(cali_aoi.total_bounds[1]), float(cali_aoi.total_bounds[3])]
aoi_lon = [float(cali_aoi.total_bounds[0]), float(cali_aoi.total_bounds[2])]
# Notice that the longitude values have negative numbers
# we need these values in a global crs so we can subtract from 360
aoi_lat, aoi_lon

The netcdf files use a global lat/lon rather than positive and negative
longitude values. To ensure you are subsetting the data for the correct
region, you can add 360 degrees to each longitude value which represent the 
min x and max x values for the extent.


In [ ]:
# The netcdf files use a global lat/lon so adjust values accordingly
aoi_lon[0] = aoi_lon[0] + 360
aoi_lon[1] = aoi_lon[1] + 360
aoi_lon

Once you have your data AOI defined, you can slice out the 
AOI region using xarray `slice`.

In [ ]:
# Slice the data by time and spatial extent
start_date = "2010-01-15"
end_date = "2010-02-15"

two_months_cali = monthly_forecast_temp_xr["air_temperature"].sel(
    time=slice(start_date, end_date),
    lon=slice(aoi_lon[0], aoi_lon[1]),
    lat=slice(aoi_lat[0], aoi_lat[1]))
two_months_cali

In [ ]:
# Plot a quick histogram
two_months_cali.plot()
plt.show()

Remember that these data are spatial. Below you 
plot each time step as a raster dataset. Notice that this spatial extent
is a rectangle representing the entire rectangular extent for the 
state of California.

In [ ]:
# Spatial Plot for the selected AOI (California)
two_months_cali.plot(col='time',
                     col_wrap=1)

plt.show()

In [ ]:
# Only subset by location / not time
cali_ts = monthly_forecast_temp_xr["air_temperature"].sel(
    lon=slice(aoi_lon[0], aoi_lon[1]),
    lat=slice(aoi_lat[0], aoi_lat[1]))
cali_ts

Calculate a summary of max temperature over time for california

In [ ]:
# Time series plot of max temperature per year. for California
# You will get a RuntimeWarning warning here because of nan values...
# This is the max value in each pixel across all months for each year
cali_annual_max = cali_ts.groupby('time.year').max(skipna=True)
cali_annual_max

### Calculate Annual Summary Data
You can calculate summary values for your aoi using the `.groupby`
method. Below you calculate the max value for each raster in the 
time series 

`.max(["lat", "lon"]` tells xarray to calculate the max on the entire raster.

the data are grouped by year. 


In [ ]:
cali_annual_max_val = cali_annual_max.groupby("year").max(["lat", "lon"])
cali_annual_max_val

In [ ]:
# Plot the data
f, ax = plt.subplots(figsize=(12, 6))
cali_annual_max_val.plot.line(hue='lat',
                              marker="o",
                              ax=ax,
                              color="grey",
                              markerfacecolor="purple",
                              markeredgecolor="purple")
ax.set(title="Annual Max Temperature (K) in California")
plt.show()

## Subset a netcdf4 Using a Shapefile Feature or Features

Above you created a rectangular spatial subset and plotted the data. 
Sometimes however you may have an AOI that is a specific polygon boundary.
Below you will modify your subset to represent just the region of the 
state of California. You will use the `regionmask` package to create
this mask.

In [ ]:
# This is the AOI of interest. You only want to calculate summary values for
# pixels within this AOI rather the entire rectangular spatial extent.
f, ax = plt.subplots()
cali_aoi.plot(ax=ax)
ax.set(title="California AOI Subset")

plt.show()

You will use the cali_aoi object created above to first create a 
mask.

In [ ]:
cali_aoi

In [ ]:
# Create a 3d mask - this contains the true / false values identifying pixels
# inside vs outside of the mask region
cali_mask = regionmask.mask_3D_geopandas(cali_aoi,
                                         monthly_forecast_temp_xr.lon,
                                         monthly_forecast_temp_xr.lat)
cali_mask

To make processing a bit quicker, below you slice out two months of the data.

In [ ]:
# Slice out two months of data
two_months = monthly_forecast_temp_xr.sel(
    time=slice('2099-10-25', '2099-12-15'))

Next, apply the mask for just the state of California to the data. 

In [ ]:
# Apply the mask for California to the data
two_months = two_months.where(cali_mask)
two_months

Below you can see that the data for only the region of interest - 
California, is plotted. However, you have a lot of extra white space
surrounding the data.

In [ ]:
two_months["air_temperature"].plot(col='time',
                                   col_wrap=1,
                                   figsize=(10, 10))
plt.show()

If you slice our your data by time and lat lon and apply the mask for your 
AOI you get the d

In [ ]:
two_months_masked = monthly_forecast_temp_xr["air_temperature"].sel(time=slice('2099-10-25',
                                                                               '2099-12-15'),
                                                                    lon=slice(aoi_lon[0],
                                                                              aoi_lon[1]),
                                                                    lat=slice(aoi_lat[0],
                                                                              aoi_lat[1])).where(cali_mask)
two_months_masked.dims

In [ ]:
two_months_masked.plot(col='time', col_wrap=1)
plt.show()

## Workflow with Multiple Regions

In the example above, you had one region of interest. Below you will implement a 
workflow where you have multiple regions. For this example, you will subset multiple states
from your shapefile. 

In [ ]:
# Start by extracting a few states from the states_gdf
states_gdf["name"]

cali_or_wash_nev = states_gdf[states_gdf.name.isin(
    ["California", "Oregon", "Washington", "Nevada"])]
cali_or_wash_nev.plot()
plt.show()

Next, create a mask just like you did above. However this time
you will use the new extent that contains 4 states (4 regions). 

In [ ]:
west_mask = regionmask.mask_3D_geopandas(cali_or_wash_nev,
                                         monthly_forecast_temp_xr.lon,
                                         monthly_forecast_temp_xr.lat)
west_mask

Below is a small helper function that grabs the AOI from a shapefile and 
returns a dictionary that you can use to slice your data. 

In [ ]:
def get_aoi(shp, world=True):
    """Takes a geopandas object and converts it to a lat/ lon
    extent 

    Parameters
    -----------
    shp : geopandas object
    world : boolean

    Returns
    -------
    Dictionary of lat and lon spatial bounds
    """

    lon_lat = {}
    # Get lat min, max
    aoi_lat = [float(shp.total_bounds[1]), float(shp.total_bounds[3])]
    aoi_lon = [float(shp.total_bounds[0]), float(shp.total_bounds[2])]

    if world:
        aoi_lon[0] = aoi_lon[0] + 360
        aoi_lon[1] = aoi_lon[1] + 360
    lon_lat["lon"] = aoi_lon
    lon_lat["lat"] = aoi_lat
    return lon_lat


west_bounds = get_aoi(cali_or_wash_nev)

Similar to your workflow above, you can subset that data as you wish by time and 
extent. Subsetting the data will make all of your data processing faster!

In [ ]:
# Slice the data
start_date = "2010-01-15"
end_date = "2010-02-15"

# Subset
two_months_west_coast = monthly_forecast_temp_xr["air_temperature"].sel(
    time=slice(start_date, end_date),
    lon=slice(west_bounds["lon"][0], west_bounds["lon"][1]),
    lat=slice(west_bounds["lat"][0], west_bounds["lat"][1]))
two_months_west_coast

Plot the data.

In [ ]:
two_months_west_coast.plot(col="time",
                           col_wrap=1,
                           figsize=(5, 8))
plt.show()

Again following the same steps as above, you can apply your
spatial mask. This will return a new array with the air
temperature data that can be grouped by each of the 4 regions. 

In [ ]:
# Apply the mask for the west coast subset
two_months_west_coast = two_months_west_coast.where(west_mask)
two_months_west_coast.dims

By default region mask takes the index of the geopandas 
object and uses it as a region identifier. If you remember 
from above, california was 53.

In [ ]:
cali_or_wash_nev

In [ ]:
# Note that the region values align with the index of the geodataframe above
two_months_west_coast.region

Having an additional region dimension in your array makes it easier
to summarize your data by region. Below you plot the 
data by time and region in a matrix. 

In [ ]:
two_months_west_coast.plot(col="time",
                           row="region",
                           sharey=False, sharex=False)
plt.show()

## Calculate Mean Air Temperature by Region
Once you have your data subsetted by region, you can 
calculate summary statistics on the data for each region. 

Below, you calculate the mean air temperature value for the 
time subset that you created above. At this point you could export
the dataframe to a csv file if you want!

In [ ]:
summary = two_months_west_coast.groupby("time").mean(["lat", "lon"])
summary.to_dataframe()

## The Entire Workflow To Subset netcdf 4 Files by Temporal and Spatial Extents

Putting all of the above together, below is an example workflow that you might use to summarize
climate data in netcdf 4 format by region and over time. 

In [ ]:
# Helper Function to extract AOI
def get_aoi(shp, world=True):
    """Takes a geopandas object and converts it to a lat/ lon
    extent """

    lon_lat = {}
    # Get lat min, max
    aoi_lat = [float(shp.total_bounds[1]), float(shp.total_bounds[3])]
    aoi_lon = [float(shp.total_bounds[0]), float(shp.total_bounds[2])]

    # Handle the 0-360 lon values
    if world:
        aoi_lon[0] = aoi_lon[0] + 360
        aoi_lon[1] = aoi_lon[1] + 360
    lon_lat["lon"] = aoi_lon
    lon_lat["lat"] = aoi_lat
    return lon_lat

In [ ]:
# Here is the entire workflow from start to end

# Download natural earth data which contains state boundaries to generate AOI
et.data.get_data(
    url="https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/cultural/ne_50m_admin_1_states_provinces_lakes.zip")

states_path = "earthpy-downloads/ne_50m_admin_1_states_provinces_lakes"
states_path = os.path.join(
    states_path, "ne_50m_admin_1_states_provinces_lakes.shp")

states_gdf = gpd.read_file(states_path)


# Get netcdf file
data_path_monthly = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_macav2metdata_tasmax_BNU-ESM_r1i1p1_rcp45_2006_2099_CONUS_monthly.nc'

# Open up the data
with xr.open_dataset(data_path_monthly) as file_nc:
    monthly_forecast_temp_xr = file_nc

# View xarray object
monthly_forecast_temp_xr

# Start by extracting a few states from the states_gdf
# I am using this as a shapefile because i presume some of my students
# will want to use custom aoi's so i want to teach using a shapefil rather than
# the regionmask built in shapes as nice as they are :)
states_gdf["name"]

cali_or_wash_nev = states_gdf[states_gdf.name.isin(
    ["California", "Oregon", "Washington", "Nevada"])]

west_mask = regionmask.mask_3D_geopandas(cali_or_wash_nev,
                                         monthly_forecast_temp_xr.lon,
                                         monthly_forecast_temp_xr.lat)
west_mask


west_bounds = get_aoi(cali_or_wash_nev)


# Slice the data
start_date = "2010-01-15"
end_date = "2020-02-15"

# Subset the data - this is now a dataarray rather than a DataSet
two_months_west_coast = monthly_forecast_temp_xr["air_temperature"].sel(
    time=slice(start_date, end_date),
    lon=slice(west_bounds["lon"][0], west_bounds["lon"][1]),
    lat=slice(west_bounds["lat"][0], west_bounds["lat"][1])).where(west_mask)

# This output is a dataarray
two_months_west_coast

# Plot the data - this produces a single histogram
two_months_west_coast.plot()
plt.show()

Once the data are subsetted, you can begin to create plots and dataframes
with the summary data. 

In [ ]:
# Keep in mind that as you add more data to your slice, the processing will slow down.
regional_summary = two_months_west_coast.groupby("region").mean(["lat", "lon"])
regional_summary.plot(col="region",
                      marker="o",
                      color="grey",
                      markerfacecolor="purple",
                      markeredgecolor="purple",
                      col_wrap=2)
plt.show()

Export to a dataframe.

In [ ]:
two_months_west_coast.groupby("region").mean(["lat", "lon"]).to_dataframe()